In [2]:
import sWeightsEff
import matplotlib.pyplot as plt

# mplhep.style.use("LHCb2")
import modplot
import numpy as np 
import pandas as pd
import uproot

In [3]:
D0masMin = 1.75
D0masMax = 1.95
D0ptMin = 1.0
D0ExpCut = f"(D0mass > {D0masMin}) & (D0mass<{D0masMax}) & (D0_pT>{D0ptMin})"

# centralityBins=[0,10]   # corresponds to 0 bin
# centralityBins=[10,40] # corresponds to 1 bin
centralityBins=[40,80] # corresponds to 2 bin

# exp_tree       = uproot.open("/home/prozorov/dev/star/OUTPUT_2014_20012024.root:Jets") # full jets
exp_tree       = uproot.open("/home/prozorov/dev/star/2014_text_trackbased_notall.root:Jets")# track-based  jets



In [4]:

centralityCuts = ["(Centrality >= {}) & (Centrality < {})".format(centralityBins[i], centralityBins[i+1]) for i in range(len(centralityBins)-1)]
# make centrality dependent analysis
print("Processing centrality bin: ", centralityCuts[0])
exp = exp_tree.arrays(["D0_pT","D0mass", "D0_r", "centrality", "centr_weight", "jet_pt_corr", "pseudorapidity",
                       "z", "lambda_1_1","lambda_1_1half","lambda_1_2","lambda_1_3", "NConst", "lambda_1_0half", "lambda_2_0", "bg_dens", "jet_area"], 
                       cut=D0ExpCut + "& (abs(pseudorapidity) <= 0.6)", library='pd')
#apply centrality cut
mapCentrality = {0:70, 1:60, 2:50, 3:40, 4:30, 5:20, 6:10, 7:5, 8:0}
exp["centrality"]  = exp["centrality"].apply(lambda x: mapCentrality[x])
jetFilter =(exp["centrality"]>=centralityBins[0]) & (exp["centrality"]<centralityBins[1])
exp = exp.loc[jetFilter]
exp["SWeight"] = sWeightsEff.getSWeights(exp.copy())
exp["Eff"] = exp.apply(lambda row: sWeightsEff.getEfficiencyD0Yield(row["D0_pT"], row["centrality"]), axis=1)
exp["Weight"] = exp["SWeight"]*exp["Eff"]*exp["centr_weight"]

exp.drop(columns=["SWeight", "Eff", "centr_weight", "D0_pT","D0mass", "centrality"], inplace=True)
exp.rename(columns={"jet_pt_corr": "pt","pseudorapidity":"eta", "NConst": "nconst", "bg_dens": "rho", "jet_area": "area", "lambda_1_0half" : "lambda_1_half" 
                     }, inplace=True)

exp ['dispersion'] = np.sqrt(exp['lambda_2_0'])

print("Data events: ", len(exp))
print(exp.head())

Processing centrality bin:  (Centrality >= 40) & (Centrality < 80)


NameError: name 'compute_sweights' is not defined